In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number, monotonically_increasing_id, when
from pyspark.sql.window import Window

# Tworzymy sesję Spark
spark = SparkSession.builder.appName("FASTQ_Reader_with_Types").getOrCreate()

# Ścieżka do katalogu z FASTQ
fastq_dir = "/Volumes/databrics_2/default/fastq"

# Lista plików FASTQ
files = ["SRR16356246_1.fastq", "SRR16356246_2.fastq"]

for file_name in files:
    file_path = f"{fastq_dir}/{file_name}"
    
    # Wczytanie pliku do DataFrame Spark
    lines_df = spark.read.text(file_path)
    
    # 1. Dodanie ciągłego numeru linii (line_number)
    window = Window.orderBy(monotonically_increasing_id())
    lines_with_id = lines_df.withColumn("line_number", row_number().over(window) - 1)
    
    # 2. Obliczenie read_id (każdy odczyt = 4 linie)
    lines_with_id = lines_with_id.withColumn("read_id", (col("line_number") / 4).cast("integer"))
    
    # 3. Określenie typu linii FASTQ (header, sequence, separator, quality)
    lines_typed = lines_with_id.withColumn(
        "line_type",
        when(col("line_number") % 4 == 0, "header")
        .when(col("line_number") % 4 == 1, "sequence")
        .when(col("line_number") % 4 == 2, "separator")
        .when(col("line_number") % 4 == 3, "quality")
    )
    
    # 4. Wyświetlenie pierwszych 12 linii z typem
    print(f"\nPlik: {file_name} - pierwsze 12 linii z line_number i line_type:")
    lines_typed.show(12, truncate=False)
    
    # 5. Liczba linii i odczytów
    total_lines = lines_typed.count()
    total_reads = total_lines // 4
    print(f"Liczba linii: {total_lines}")
    print(f"Liczba odczytów: {total_reads}\n")



Plik: SRR16356246_1.fastq - pierwsze 12 linii z line_number i line_type:


/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+-----+-----------+-------+---------+
|value|line_number|read_id|line_type|
+-----+-----------+-------+---------+
+-----+-----------+-------+---------+

Liczba linii: 0
Liczba odczytów: 0


Plik: SRR16356246_2.fastq - pierwsze 12 linii z line_number i line_type:
+-----+-----------+-------+---------+
|value|line_number|read_id|line_type|
+-----+-----------+-------+---------+
+-----+-----------+-------+---------+

Liczba linii: 0
Liczba odczytów: 0

